# Assignment 3
## s3843790

In [1]:
import pandas as pd
import numpy as np

# Load MovieLens 100K dataset into a dataframe of pandas
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data', sep='\t', names=names)
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [2]:
# Select 500 most active users and 500 most active items from the dataset
n_most_active_users = 500
n_most_active_items = 500

user_ids = df.groupby('user_id').count().sort_values(by='rating', ascending=False).head(n_most_active_users).index
item_ids = df.groupby('item_id').count().sort_values(by='rating', ascending=False).head(n_most_active_items).index
df = df[(df['user_id'].isin(user_ids)) & (df['item_id'].isin(item_ids))]

In [3]:
# Map new internal ID for items
i_ids = df['item_id'].unique().tolist()
item_dict = dict(zip(i_ids, [i for i in range(len(i_ids))]))
df['item_id'] = df['item_id'].map(item_dict)

# Split Dataset

In [4]:
# The number of training users and active users
n_training_users = 300
n_active_users = n_most_active_users - n_training_users

# The number of GIVEN ratings for active users
GIVEN = 20

# Randomly select users from the most active users as training set
random_uids = np.random.choice(df.user_id.unique(), n_training_users, replace=False)
train_df = df[df['user_id'].isin(random_uids)]
# Map new internal ID for all users in the training set
u_ids = train_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
train_df['user_id'] = train_df['user_id'].map(user_dict)

# The rest of users are active users for testing
remain_df = df[~df['user_id'].isin(random_uids)]
# Map new internal ID for all active users
u_ids = remain_df['user_id'].unique().tolist()
user_dict = dict(zip(u_ids, [i for i in range(len(u_ids))]))
remain_df['user_id'] = remain_df['user_id'].map(user_dict)

# Randomly select GIVEN ratings for active users
active_df = remain_df.groupby('user_id').sample(n=GIVEN, random_state=1024)

test_df = remain_df[~remain_df.index.isin(active_df.index)]

<ipython-input-4-2ff1e0f0649f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['user_id'] = train_df['user_id'].map(user_dict)
<ipython-input-4-2ff1e0f0649f>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  remain_df['user_id'] = remain_df['user_id'].map(user_dict)


In [5]:
# Convert the format of datasets to matrices
df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_training_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_training_users), 'rating': 0})
train_ds = df_zeros.merge(train_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

df_zeros = pd.DataFrame({'user_id': np.tile(np.arange(0, n_active_users), n_most_active_items), 'item_id': np.repeat(np.arange(0, n_most_active_items), n_active_users), 'rating': 0})
active_ds = df_zeros.merge(active_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')
test_ds = df_zeros.merge(test_df, how='left', on=['user_id', 'item_id']).fillna(0.).pivot_table(values='rating_y', index='user_id', columns='item_id')

train_ds, active_ds, test_ds

(item_id  0    1    2    3    4    5    6    7    8    9    ...  490  491  492  \
 user_id                                                    ...                  
 0        0.0  0.0  4.0  4.0  4.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  0.0   
 1        4.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 2        4.0  0.0  5.0  0.0  0.0  3.0  4.0  2.0  0.0  2.0  ...  0.0  2.0  0.0   
 3        4.0  0.0  5.0  0.0  1.0  0.0  3.0  2.0  0.0  0.0  ...  0.0  0.0  0.0   
 4        3.0  0.0  4.0  0.0  0.0  3.0  2.0  2.0  0.0  5.0  ...  0.0  4.0  0.0   
 ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
 295      0.0  0.0  5.0  4.0  0.0  1.0  0.0  0.0  0.0  1.0  ...  0.0  0.0  0.0   
 296      4.0  0.0  3.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 297      0.0  0.0  0.0  5.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
 298      0.0  0.0  0.0  3.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0  5.0   
 299      0.0  0

In [6]:
# Predicting All Missing Data in training set
imputed_train_ds = train_ds.values.copy()

# Your implementation to predict the missing values
(Put all your implementation for your algorithm in the following cell only to handle the missing values; )

In [7]:
## Put all your implementation for your solution in this cell only to predict the missing values; 
## NOTE 1: DO NOT change anything in the rest of the cells in this framework, 
## otherwise the changes might cause errors and make your implementation invalid.

## Note 2: 
## The user-item rating matrix is imputed_train_ds, 
## and the missing values are those 0s in imputed_train_ds. 
## You are required to predict them by using the solution in the given report. 

## The following parameters are required in the given report, 
## which is named "Effective Missing Data Prediction for Collaborative Filtering", 
## and you will need to use them. But, please do not change their values. 
LAMBDA = 0.7    # λ
GAMMA = 10      # γ
DELTA = 10      # δ
ITA = 0.7       # η
THETA = 0.7     # θ
EPSILON = 1e-9
imputed_train_ds_copy=imputed_train_ds.copy()

def Sim_U(a,u,GAMMA):
    U_a=imputed_train_ds[a,:] #extract the data vector of a 'th' user
    U_u=imputed_train_ds[u,:] #extract the data vector of u 'th' user
    inters=U_a*U_u!=0 # intersection of above two sets
    U_a=U_a[inters] # select common items
    mean_ra=np.mean(U_a)
    ra=U_a-mean_ra
    U_u=U_u[inters]# select common items
    mean_ru=np.mean(U_u)
    ru=U_u-mean_ru
    return((min(sum(inters),GAMMA)/GAMMA)*np.sum(ra*ru)/(np.sqrt(np.sum(ra*ra))*np.sqrt(np.sum(ru*ru))))

def Sim_I(i,j,DELTA):
    I_i=imputed_train_ds[:,i] #extract the data vector of i 'th' item
    I_j=imputed_train_ds[:,j] #extract the data vector of j 'th' item
    inters=I_i*I_j!=0 # intersection of above two set
    I_i=I_i[inters] # select common users
    mean_ri=np.mean(I_i)
    ri=I_i-mean_ri
    I_j=I_j[inters] # select common users
    mean_rj=np.mean(I_j)
    rj=I_j-mean_rj
    return((min(sum(inters),DELTA)/DELTA)*np.sum(ri*rj)/(np.sqrt(np.sum(ri*ri))*np.sqrt(np.sum(rj*rj))))

## Find similar users
def Sim_users(u_indx,GAMMA,ITA):
    indx_array=[]
    for i in range(imputed_train_ds.shape[0]):
        if i!=u_indx:
            if Sim_U(i,u_indx,GAMMA)>ITA: #Check the condition
                indx_array.append(i)
    return(indx_array)

## Find similar items
def Sim_items(i_indx,DELTA,THETA):
    indx_array=[]
    for i in range(imputed_train_ds.shape[1]):
        if i!=i_indx:
            if Sim_I(i,i_indx,DELTA)>THETA: #Check the condition
                indx_array.append(i)
    return(indx_array)

def P_u(u_indx,i_indx,GAMMA):
    S_u=Sim_users_array[u_indx] # Find similar users
    Sum_numerator=0
    Sum_denominator=0
    for u in S_u:
        Sum_numerator+=Sim_U(u_indx,u,GAMMA)*(imputed_train_ds[u,i_indx]-np.mean(imputed_train_ds[u,:]))# Find the sum of numerator
        Sum_denominator+=Sim_U(u_indx,u,GAMMA) # Find the sum of denominator
    return(np.mean(imputed_train_ds[u_indx,:])+(Sum_numerator/Sum_denominator))

def P_i(u_indx,i_indx,DELTA):
    S_i=Sim_items_array[i_indx] # Find similar items
    Sum_numerator=0
    Sum_denominator=0
    for i in S_i:
        Sum_numerator+=Sim_I(i_indx,i,DELTA)*(imputed_train_ds[u_indx,i]-np.mean(imputed_train_ds[:,i])) # Find the sum of numerator
        Sum_denominator+=Sim_I(i_indx,i,DELTA) # Find the sum of denominator
    return(np.mean(imputed_train_ds[:,i_indx])+(Sum_numerator/Sum_denominator))

## Main function to calculate missing values
def P_ui(u_indx,i_indx,LAMBDA,DELTA,GAMMA):
    Similar_users=Sim_users_array[u_indx]
    Similar_items=Sim_items_array[i_indx]
    if len(Similar_users)==0 and len(Similar_items)==0:
        P=0
    elif len(Similar_users)==0:
        P=P_i(u_indx,i_indx,DELTA)
    elif len(Similar_items)==0:
        P=P_u(u_indx,i_indx,GAMMA)
    else:
        P=LAMBDA*P_u(u_indx,i_indx,GAMMA)+(1-LAMBDA)*P_i(u_indx,i_indx,DELTA)
    return(P)

## Find similar users for each individual user
Sim_users_array=[]
for u_indx in range(imputed_train_ds.shape[0]):
        Sim_users_array.append(Sim_users(u_indx,GAMMA,ITA)) #append similar users to an array

## Find similar items for each individual item
Sim_items_array=[]
for i_indx in range(imputed_train_ds.shape[1]):
        Sim_items_array.append(Sim_items(i_indx,DELTA,THETA)) #append similar items to an array

for u_indx in range(imputed_train_ds.shape[0]):
    for i_indx in range(imputed_train_ds.shape[1]):
        if imputed_train_ds[u_indx,i_indx]==0:
            imputed_train_ds_copy[u_indx,i_indx]=P_ui(u_indx,i_indx,LAMBDA,DELTA,GAMMA) #update missing values
    # print(str(u_indx+1)+"/"+str(imputed_train_ds.shape[0])+" completed")  # used to check that program works & didn't
    # freeze since this process takes a very long time. Not necessary for the code to run properly.
imputed_train_ds=imputed_train_ds_copy

<ipython-input-7-6f662bf887d1>:31: RuntimeWarning: invalid value encountered in double_scalars
  return((min(sum(inters),GAMMA)/GAMMA)*np.sum(ra*ru)/(np.sqrt(np.sum(ra*ra))*np.sqrt(np.sum(ru*ru))))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<ipython-input-7-6f662bf887d1>:43: RuntimeWarning: invalid value encountered in double_scalars
  return((min(sum(inters),DELTA)/DELTA)*np.sum(ri*rj)/(np.sqrt(np.sum(ri*ri))*np.sqrt(np.sum(rj*rj))))


# Evaluation

### Compute Pearson Correlation Coefficient of All Pairs of Items between active set and imputed training set

In [8]:
imputed_train_ds = pd.DataFrame(imputed_train_ds)
imputed_train_ds

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.709742,0.185840,4.000000,4.000,4.000000,0.000000,0.615872,4.000000,0.713909,0.000000,...,0.436710,-0.290000,-0.397068,-0.085740,-0.275216,-0.170000,-0.637992,0.497573,0.970816,-0.324074
1,4.000000,-0.120412,0.842319,2.000,0.279745,1.191273,1.288264,1.724578,-0.215410,3.267755,...,-0.045155,0.386986,-0.295284,0.123189,0.335312,0.269572,-0.030168,0.795629,-0.348324,-0.260279
2,4.000000,0.185840,5.000000,0.000,-0.065279,3.000000,4.000000,2.000000,-0.130822,2.000000,...,0.478999,2.000000,0.631335,-0.085740,0.858389,-0.170000,-0.399509,4.000000,0.073221,0.090053
3,4.000000,0.185840,5.000000,0.000,1.000000,0.000000,3.000000,2.000000,0.661972,0.000000,...,-0.142872,3.710000,-0.397068,4.000000,1.000000,-0.170000,-0.637992,2.340809,-0.573869,2.000000
4,3.000000,0.185840,4.000000,0.000,-0.065279,3.000000,2.000000,2.000000,0.076659,5.000000,...,0.366388,4.000000,-0.072927,2.096627,0.582543,-0.170000,0.173373,0.302641,-0.573869,0.337994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.709742,0.503261,5.000000,4.000,-0.065279,1.000000,0.615872,0.986667,-0.130822,1.000000,...,0.356034,-0.290000,-0.397068,-0.085740,0.460026,0.830000,0.149024,-0.272173,1.000000,0.874082
296,4.000000,0.288152,3.000000,0.332,0.688309,0.332000,0.417162,1.428400,0.193153,0.332000,...,0.189538,0.145400,0.627481,0.206678,0.484721,2.281400,0.376466,0.317055,0.060239,0.128226
297,0.177945,0.020774,0.288023,5.000,0.579429,-0.049968,0.149784,2.121520,0.179195,2.607886,...,0.294261,-0.121977,1.476403,0.484591,-0.117542,1.030321,1.493025,-0.116629,-0.207138,-0.118913
298,0.709742,0.185840,1.076667,3.000,-0.065279,0.000000,0.615872,4.000000,0.291544,0.000000,...,-0.142872,-0.290000,5.000000,-0.085740,0.131415,3.000000,-0.280022,0.097397,0.018340,-0.835702


In [9]:
active_user_pearson_corr = np.zeros((active_ds.shape[0], train_ds.shape[0]))

# Compute Pearson Correlation Coefficient of All Pairs of Users between active set and imputed training set
for i, user_i_vec in enumerate(active_ds.values):
    for j, user_j_vec in enumerate(imputed_train_ds.values):
        
        # ratings corated by the current pair od users
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0

        # corrated item index, skip if there are no corrated ratings
        corrated_index = np.intersect1d(np.where(mask_i), np.where(mask_j))
        if len(corrated_index) == 0:
            continue

        # average value of user_i_vec and user_j_vec
        mean_user_i = np.sum(user_i_vec) / (np.sum(np.clip(user_i_vec, 0, 1)) + EPSILON)
        mean_user_j = np.sum(user_j_vec) / (np.sum(np.clip(user_j_vec, 0, 1)) + EPSILON)

        # compute pearson corr
        user_i_sub_mean = user_i_vec[corrated_index] - mean_user_i
        user_j_sub_mean = user_j_vec[corrated_index] - mean_user_j

        r_ui_sub_r_i_sq = np.square(user_i_sub_mean)
        r_uj_sub_r_j_sq = np.square(user_j_sub_mean)

        r_ui_sum_sqrt = np.sqrt(np.sum(r_ui_sub_r_i_sq))
        r_uj_sum_sqrt = np.sqrt(np.sum(r_uj_sub_r_j_sq))

        sim = np.sum(user_i_sub_mean * user_j_sub_mean) / (r_ui_sum_sqrt * r_uj_sum_sqrt + EPSILON)

        # significance weighting
        weighted_sim = (min(len(corrated_index), GAMMA) / GAMMA) * sim

        active_user_pearson_corr[i][j] = weighted_sim

active_user_pearson_corr

array([[-0.38303595, -0.23134493, -0.09747023, ..., -0.15247848,
        -0.20815069, -0.65427675],
       [-0.18746657, -0.00124785,  0.29575709, ..., -0.44828989,
        -0.80511005, -0.15298635],
       [ 0.14528534,  0.20069997,  0.60650224, ...,  0.39705794,
         0.50926306, -0.1200081 ],
       ...,
       [ 0.16916139,  0.13498877,  0.36317678, ...,  0.01153526,
         0.29426612,  0.04629905],
       [ 0.3238444 ,  0.47840437,  0.19637531, ...,  0.15209106,
        -0.09605882,  0.33775953],
       [ 0.13064772,  0.2434322 ,  0.26074242, ...,  0.36882501,
         0.27123087,  0.13553229]])

## Predict Ratings of Testing Set

In [10]:
K = 10

test_ds_pred = np.zeros_like(test_ds.values)

for (i, j), rating in np.ndenumerate(test_ds.values):

    if rating > 0:

        sim_user_ids = np.argsort(active_user_pearson_corr[i])[-1:-(K + 1):-1]

        #==================user-based==================#
        # the coefficient values of similar users
        sim_val = active_user_pearson_corr[i][sim_user_ids]

        # the average value of the current user's ratings
        sim_users = imputed_train_ds.values[sim_user_ids]
        user_mean = np.sum(active_ds.values[i]) / (np.sum(np.clip(active_ds.values[i], 0, 1)) + EPSILON)
        sim_user_mean = np.sum(sim_users, axis=1) / (np.sum(np.clip(sim_users, 0, 1), axis=1) + EPSILON)

        # select the users who rated item j
        mask_rated_j = sim_users[:, j] > 0
        
        # sim(u, v) * (r_vj - mean_v)
        sim_r_sum_mean = sim_val[mask_rated_j] * (sim_users[mask_rated_j, j] - sim_user_mean[mask_rated_j])
        
        user_based_pred = user_mean + np.sum(sim_r_sum_mean) / (np.sum(sim_val[mask_rated_j]) + EPSILON)
        user_based_pred = np.clip(user_based_pred, 0, 5)

        test_ds_pred[i][j] = user_based_pred
        
test_ds_pred


array([[0.        , 0.        , 0.        , ..., 0.        , 2.39155812,
        0.        ],
       [0.        , 2.15099917, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [5.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 5.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

## Compute MAE and RMSE

In [11]:
# MAE
MAE = np.sum(np.abs(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1))

# RMSE
RMSE = np.sqrt(np.sum(np.square(test_ds_pred - test_ds.values)) / np.sum(np.clip(test_ds.values, 0, 1)))

print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

MAE: 0.9161860989123678, RMSE: 1.1706436261408193
